<h1>REDfold</h1>

installed package but did not acknowledge how to use it (I need fasta data to train model, the data provided by developer is in .ct format, should I train on my own data or not), downloaded the database used by developers but it is in .ct format and very big and filenames somehow contain dots, so I will try to benchmark it on short dna aptamers dataset 

In [1]:
import pandas as pd

import time
from selenium import webdriver
from selenium.webdriver.common.by import By

from lxml import etree
from bs4 import BeautifulSoup


import subprocess

In [2]:
def redfold_web(s):

    s = s.replace('t', 'u')
    s = s.replace('T', 'U')

    driver = webdriver.Chrome()
    # Open web-server
    driver.get('https://redfold.ee.ncyu.edu.tw/redfold/')  # Replace with your target URL

    # type sequence
    input_seq = driver.find_element(By.ID, "Text1")
    input_seq.send_keys(s)

    submitbutton = driver.find_element(By.NAME, 'Submit')
    submitbutton.click()

    result_html = driver.page_source
    driver.quit()

    soup = BeautifulSoup(result_html, 'html.parser')
    # Находим элемент <pre> по классу
    pre_element = soup.find('font')
    text = pre_element.get_text()
    text = text.split()

    # Извлекаем текст из элемента
    return text[4]

redfold_web('aaaaaaaaaaaaatttttttttttt')

'.........................'

In [3]:
aptabase = pd.read_csv('Aptamers_2D.csv')
seqs = list(aptabase['Sequence'])
preds = []
for s in seqs:
    preds.append(redfold_web(s))
    print(len(preds), end='-')

1-2-3-4-5-6-7-8-9-10-11-12-13-14-15-16-17-18-19-20-21-22-23-24-25-26-27-28-29-30-31-32-33-34-35-36-37-38-39-40-41-42-43-44-45-46-47-48-49-50-51-52-53-54-55-56-57-58-59-60-61-62-63-64-65-66-67-68-69-

In [4]:
refs = aptabase['2D']

In [5]:
def find_basepairs(s):
    stack = []
    result = []

    for index, char in enumerate(s):
        if char == '(':
            stack.append(index)
        elif char == ')':
            # Если нашли закрывающую скобку, сопоставляем с последней открывающей
            if stack:
                open_index = stack.pop()
                result.append((open_index, index))

    return sorted(result)

def find_knotpairs(s):
    stack = []
    #result = {}
    result = []

    for index, char in enumerate(s):
        if char == '[':
            # Запоминаем индекс открывающей скобки
            stack.append(index)
        elif char == ']':
            # Если нашли закрывающую скобку, сопоставляем с последней открывающей
            if stack:
                open_index = stack.pop()
                #result[open_index] = index
                result.append((open_index, index))

    return sorted(result)

def find_otherpairs(s):
    stack = []
    #result = {}
    result = []

    for index, char in enumerate(s):
        if char == '{':
            # Запоминаем индекс открывающей скобки
            stack.append(index)
        elif char == '}':
            # Если нашли закрывающую скобку, сопоставляем с последней открывающей
            if stack:
                open_index = stack.pop()
                #result[open_index] = index
                result.append((open_index, index))

    return sorted(result)

def find_pairs(s):
    basepairs = find_basepairs(s)
    knotpairs = find_knotpairs(s)
    otherpairs = find_otherpairs(s)
    pairs = basepairs + knotpairs + otherpairs
    return sorted(pairs)



# def possible_pairs(s):

#     l = len(s)
#     pairs = []
#     for i in range(l):
#         for j in range(i+1,l):
#             pair = (i, j)
#             pairs.append(pair)

#     return pairs

#faster benchmark which do not calculate accuracy and TN
def Full_Benchmark_2Dv2(pred, ref, v=True):
    
    #Taminoto accuracy, adequate in case of g-quadruplexes predicted as '+' but not '[' and ']'
    tcounter = 0
    for i in range(len(ref)):
        if ref[i] == pred[i]:
            tcounter += 1

        elif (ref[i] != '.') and pred[i] == '+':
            tcounter += 1
    taminoto = tcounter/len(ref)


    #Binary classification metrics for all pairs
    pred_pairs = find_pairs(pred)
    ref_pairs = find_pairs(ref)
    # poss_pairs = possible_pairs(ref)
    
    # ref_notpairs = [x for x in poss_pairs if x not in ref_pairs] 
    # pred_notpairs = [x for x in poss_pairs if x not in pred_pairs]
    

    TP = len(set(pred_pairs).intersection(ref_pairs))
    FP = len(pred_pairs)-TP
    # TN = len(set(pred_notpairs).intersection(ref_notpairs)) 
    # FN = len(pred_notpairs)-TN
    FN = len(ref_pairs) -TP

    if TP == 0:
        TP = 0.0001

    if FP == 0:
        FP = 0.0001

    # if TN == 0:
    #     TN = 0.0001

    if FN == 0:
        FN = 0.0001
    
    # print(TP, TN, FP, FN, TP+FP+TN+FN)
    # print(len(pred_pairs), len(pred_notpairs), len(ref_pairs), len(ref_notpairs), len(poss_pairs))

    # Accuracy = (TP+TN)/(TP+TN+FP+FN)
    Precision = TP/(TP+FP)
    Recall = TP/(TP+FN)
    F1_score = (2*Precision*Recall)/(Precision+Recall)

    
    if v:
        print('Taminoto accuracy = ', round(taminoto, 3))
        # print('Accuracy =', round(Accuracy, 3))
        print('Precision =', round(Precision, 3))
        print('Recall =', round(Recall, 3))
        print('F1-score =', round(F1_score, 3))

    else:
        return taminoto, Precision, Recall, F1_score

#Taminoto accuracy, adequate in case of g-quadruplexes predicted as '+' but not '[' and ']'
def taminoto(pred, ref):
    tcounter = 0
    for i in range(len(ref)):
        if ref[i] == pred[i]:
            tcounter += 1

        elif (ref[i] != '.') and pred[i] == '+':
            tcounter += 1  

    return tcounter/len(ref)


def taminoto_accurate_percent(preds, refs, cutoff=0.85, v=True):
    
    acc_list = []
    acc = 0
    acc_counter = 0
    for i in range(len(preds)):
        acc = taminoto(preds[i], refs[i])
        if acc >= cutoff:
            acc_counter += 1
        acc_list.append(acc)

    if v:
        print(f'% of Accurate structures (taminoto acc. >= {cutoff}) =', round(100*acc_counter/len(preds), 2))

    else:
        return acc_counter/len(preds), acc_list

Dataset used by developers to test model

In [6]:

ref = ''.join(refs)
pred = ''.join(preds)

Full_Benchmark_2Dv2(ref, pred)

Taminoto accuracy =  0.315
Precision = 0.069
Recall = 0.789
F1-score = 0.128


In [7]:
taminoto_accurate_percent(preds, refs)

% of Accurate structures (taminoto acc. >= 0.85) = 0.0


In [8]:
aptabase['REDfold'] = preds
aptabase.to_csv('REDfold_benchmarking.csv')